# 📄 PDF Translator Enhanced v2.0 - Colab Edition

**Translate scientific PDFs with 100% formula preservation**

> 🔗 **GitHub:** [error-wtf/pdf-translator-enhanced](https://github.com/error-wtf/pdf-translator-enhanced)  
> Based on [thelanguagenerd/pdf-translator](https://github.com/thelanguagenerd/pdf-translator)

© 2025 Sven Kalinowski with small help of Lino Casu  
Licensed under the Anti-Capitalist Software License v1.4

---

## 🆕 v2.0 Features

- **🔬 100% Formula Preservation** - Hash-based isolation, no corruption
- **🌍 25 Languages** - Including CJK, RTL (Arabic, Hebrew), Cyrillic
- **📊 Quality Scoring** - Automatic 0-100 evaluation
- **📄 Sequential Processing** - Handles PDFs of ANY size (page-by-page)
- **💾 Caching** - Faster re-translations
- **☁️ Cloud Models** - Use huge models without local GPU!

---

## 🚀 How It Works

1. **Run all cells** - Installs Ollama, clones repo, installs dependencies
2. **Gradio launches** with a **public share link**
3. **Access from any device** - The link works for 72 hours

---

## 🤖 Available Models

### Local Models (runs on Colab GPU)
| Model | Size | Quality | VRAM |
|-------|------|---------|------|
| `gemma2:2b` | 1.6 GB | Basic | 4 GB |
| `phi3:mini` | 2.3 GB | Good | 4 GB |
| `llama3.2:3b` | 2 GB | Good | 4 GB |
| `mistral:7b` | 4.1 GB | Very Good | 8 GB |
| **`qwen2.5:7b`** ⭐ | 4.4 GB | Excellent | 8 GB |
| `llama3.1:8b` | 4.7 GB | Very Good | 10 GB |
| `openchat:7b` | 4.1 GB | Very Good | 8 GB |
| `qwen2.5:14b` | 9 GB | Premium | 16 GB |
| `deepseek-coder-v2:16b` | 9 GB | Premium | 16 GB |
| `gpt-oss:20b` | 12 GB | Premium | 16 GB |

### ☁️ Cloud Models (NO GPU needed!)
| Model | Parameters | Quality |
|-------|------------|----------|
| `gpt-oss:20b-cloud` | 20B | Good |
| `gpt-oss:120b-cloud` | 120B | Excellent |
| `qwen2.5:32b-cloud` | 32B | Premium |
| `qwen2.5:72b-cloud` | 72B | Maximum |
| `deepseek-v3:671b-cloud` | 671B | Maximum |

⚠️ Cloud models require: `ollama signin`

---

## GPU Runtime Setup

1. Click **Runtime** → **Change runtime type**
2. Select **T4 GPU** (free) or **A100** (Colab Pro)
3. Click **Save**

In [ ]:
#@title 1. Check GPU
!nvidia-smi --query-gpu=name,memory.total --format=csv
import torch
print(f"\n✅ CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    vram_gb = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"🎮 GPU: {gpu_name}")
    print(f"💾 VRAM: {vram_gb:.1f} GB")
    if vram_gb >= 16:
        print("\n📌 Recommended: qwen2.5:14b, deepseek-coder-v2:16b, or gpt-oss:20b")
    elif vram_gb >= 8:
        print("\n📌 Recommended: qwen2.5:7b or mistral:7b")
    else:
        print("\n📌 Recommended: llama3.2:3b, phi3:mini, or gemma2:2b")
else:
    print("\n❌ No GPU detected!")
    print("📌 Use ☁️ Cloud models (no GPU needed):")
    print("   - gpt-oss:120b-cloud")
    print("   - deepseek-v3:671b-cloud")

In [ ]:
#@title 2. Install Ollama
!curl -fsSL https://ollama.ai/install.sh | sh
print("\n✅ Ollama installed!")

In [ ]:
#@title 3. Start Ollama & Download Model
import subprocess
import time
import requests

# Start Ollama server
print("🚀 Starting Ollama server...")
subprocess.Popen(["ollama", "serve"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
time.sleep(5)

# Check if running
try:
    r = requests.get("http://localhost:11434/api/tags", timeout=5)
    if r.status_code == 200:
        print("✅ Ollama server running!")
except:
    print("⚠️ Ollama not responding. Run this cell again.")

# ═══════════════════════════════════════════════════════════════
# MODEL LIST
# ───────────────────────────────────────────────────────────────
# LOCAL (up to 16 GB VRAM - fits on Colab T4/A100):
#   gemma2:2b, phi3:mini, llama3.2:3b, mistral:7b, qwen2.5:7b,
#   llama3.1:8b, openchat:7b, qwen2.5:14b, deepseek-coder-v2:16b,
#   gpt-oss:20b
# ───────────────────────────────────────────────────────────────
# CLOUD (no GPU needed, requires 'ollama signin'):
#   gpt-oss:20b-cloud, gpt-oss:120b-cloud, qwen2.5:32b-cloud,
#   qwen2.5:72b-cloud, deepseek-v3:671b-cloud
# ═══════════════════════════════════════════════════════════════

model = "qwen2.5:7b"  #@param ["gemma2:2b", "phi3:mini", "llama3.2:3b", "mistral:7b", "qwen2.5:7b", "llama3.1:8b", "openchat:7b", "qwen2.5:14b", "deepseek-coder-v2:16b", "gpt-oss:20b", "gpt-oss:20b-cloud", "gpt-oss:120b-cloud", "qwen2.5:32b-cloud", "qwen2.5:72b-cloud", "deepseek-v3:671b-cloud"]

is_cloud = model.endswith("-cloud")

if is_cloud:
    print(f"\n☁️ Cloud model selected: {model}")
    print("\n⚠️ Cloud models require Ollama signin:")
    print("   Run: ollama signin")
    print("   Then run this cell again.")
else:
    # Show model size info
    model_sizes = {
        "gemma2:2b": "1.6 GB", "phi3:mini": "2.3 GB", "llama3.2:3b": "2 GB",
        "mistral:7b": "4.1 GB", "qwen2.5:7b": "4.4 GB", "llama3.1:8b": "4.7 GB",
        "openchat:7b": "4.1 GB", "qwen2.5:14b": "9 GB",
        "deepseek-coder-v2:16b": "9 GB", "gpt-oss:20b": "12 GB"
    }
    size = model_sizes.get(model, "unknown")
    print(f"\n📥 Downloading {model} ({size})...")
    print("   This may take a few minutes.")

!ollama pull {model}
print(f"\n✅ Model {model} ready!")

In [ ]:
#@title 4. Clone Repository & Install Dependencies
%cd /content
!rm -rf pdf-translator-enhanced 2>/dev/null
!git clone https://github.com/error-wtf/pdf-translator-enhanced.git
%cd pdf-translator-enhanced

print("\n📦 Installing dependencies...")
!pip install -q -r requirements.txt
print("\n✅ All dependencies installed!")

In [ ]:
#@title 5. Launch Gradio App with Public Share Link 🚀
import os
os.chdir("/content/pdf-translator-enhanced")

print("=" * 60)
print("   📄 PDF Translator v2.0 - Starting Gradio UI")
print("=" * 60)
print("")
print("🌐 A PUBLIC SHARE LINK will be generated below!")
print("📱 You can access it from any device for 72 hours.")
print("")
print("✨ v2.0 Features:")
print("   - 100% Formula Preservation")
print("   - 25 Languages (including CJK, RTL)")
print("   - Sequential Processing (handles ANY PDF size)")
print("   - Quality Scoring (0-100)")
print("   - Retry Logic for stable translations")
print("   - ETA & Progress Display")
print("")
print("=" * 60)

# Run the app with --share argument
!python gradio_app.py --share